In [60]:
import tkinter
gui = tkinter.Tk(className = ' Song Popularity')
gui.geometry('600x400')
gui.configure(bg = 'black')
var = StringVar()
title = Label(gui,textvariable = var, fg = 'white',bg = 'black', font = ('Times New Roman', 20))
var.set('Song Popularity')
title.pack()
songName = Label(gui,textvariable = var, fg = 'white', bg = 'black', font = ('Times New Roman', 20))
var.set('Song Name')
songName.pack(side = LEFT)
e1 = Entry(gui)
e1.pack(side = RIGHT)
gui.mainloop()

In [54]:
import tkinter
gui = tkinter.Tk(className = ' Song Popularity')
img = Canvas(gui, width = 400, height = 300,  relief = 'raised')
filename = 'ezgif.com-gif-maker.gif'
image = img.create_image(400, 300, anchor=NE, image=filename)
#gif1 = PhotoImage(file='IMG_246820211108-200923.JPG')
#canvas.create_image(50, 10, image=gif1, anchor=NW)

TclError: couldn't recognize data in image file "IMG_246820211108-200923.JPG"